# Preprocess IMERG data 

This Jupyter Notebook preprocesses the original IMERG data (30 minutes precipitation rates). The preprocessing comprises the following steps:
- merging from hourly to monthly data files
- spatial slicing for a target region 
- reordering the dimensions for consistency with the ERA5 data (and also to avoid issues with CDO)
- remapping to a regular 0.0833°-grid (same as the regridded CERRA data used for AtmoRep)

In [ ]:
import os
import glob
from pathlib import Path
import subprocess as sp
import xarray as xr
import pandas as pd
from cdo import Cdo

In [ ]:
imerg_dir_origin = Path("/p/scratch/atmo-rep/data/imerg/")
imerg_dir_target = Path("/p/scratch/atmo-rep/data/imerg_regridded/")

yr_start = 2018
yr_end = 2018

grid_des_target = "/p/project/deepacf/atmo-rep/langguth1/downstream-applications/downscaling-cgan/grid_des/cerra_dom_dx0p0833_griddes"
grid_des_input = "/p/project/deepacf/atmo-rep/langguth1/downstream-applications/downscaling-cgan/grid_des/imerg_griddes"

target_domain = [-20, 36, 24, 75]     # [lon_west, lon_east, lat_south, lat_north]

temp_dir = Path("/p/scratch/atmo-rep/cache/cdo_temp")

In [ ]:
cdo = Cdo(tempdir=temp_dir)

yr_m = pd.date_range(f"{yr_start}-01-01", pd.to_datetime(f"{yr_end}-12-31") + pd.Timedelta(7, "d"), freq="m")

In [ ]:
for m in yr_m:
    print(f"Start processing data for {m.strftime('%Y-%m')}...")
    m_str_pr = m.strftime("%Y-%m")
    m_str = m_str_pr.replace("-", "") 
    
    flist = glob.glob(str(imerg_dir_origin.joinpath(f"3B-HHR.MS.MRG.3IMERG.{m_str}*-S*.HDF5.nc4")))
    # sanity check
    nfiles_expected = m.days_in_month*24 
    nfiles_found = len(flist)
    
    if nfiles_found != nfiles_expected:
        print(f"Unexpected number of files found for {m_str_pr}. Expected {nfiles_expected}, but found {nfiles_found} files. Skip processing data for this month.")
        continue
    
    # merge data to monthly file
    outfile = imerg_dir_target.joinpath(f"3B-HHR.MS.MRG.3IMERG.{m.strftime('y%Y_m%m')}.nc")
    print("Start merging monthly data...")
    ofile = cdo.mergetime(input=flist)
    print("Slice spatially monthly data...")
    ofile_tmp = f"{ofile}_tmp.nc"
    ofile = cdo.sellonlatbox(",".join([str(num) for num in target_domain]), input=ofile)#, output=ofile_tmp)
    # re-order dimensions
    cmd_ncpdq = f"ncpdq -O --rdr=time,lat,lon {ofile} {ofile}"
    sp.check_output(cmd_ncpdq, shell=True)
    # correct dimension attribute
    cmd_ncatted = f'ncatted -O -a DimensionNames,precipitation,o,c,"time,lat,lon" {ofile}'
    sp.check_output(cmd_ncatted, shell=True)
    print("Perform remapping onto target grid...")
    cdo.remapcon(grid_des_target, input=f"-setgrid,{grid_des_input} {str(ofile)}", output=str(outfile))
    
    print(f"Processing of data for {m_str_pr} finished...")

In [ ]:
",".join([-20, 36, 24, 75])
